In [7]:
!pip install --upgrade pandas pyarrow datasets
!pip install pinecone-client sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached pyarrow-18.0.0-cp312-cp312-macosx_12_0_arm64.whl.metadata (3.3 kB)
  Using cached datasets-3.1.0-py3-none-any.whl.metadata (20 kB)
Using cached pyarrow-18.0.0-cp312-cp312-macosx_12_0_arm64.whl (29.5 MB)
Using cached datasets-3.1.0-py3-none-any.whl (480 kB)
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 17.0.0
    Uninstalling pyarrow-17.0.0:
      Successfully uninstalled pyarrow-17.0.0
  Attempting uninstall: datasets
    Found existing installation: datasets 3.0.0
    Uninstalling datasets-3.0.0:
      Successfully uninstalled datasets-3.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.37.1 requires pillow<11,>=7.1.0, but you have pillow 11.0.0 which is incompatible.
streamlit 1.37.1 requires tenacity<9,>=8.1.0, but you have tenacity 9.0.0 which is incompatible.
autorag 0.3.8 requires numpy<2.0.0, but yo

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
import os
import json
from dotenv import load_dotenv
from pinecone import Pinecone
from pinecone import ServerlessSpec
from sentence_transformers import SentenceTransformer

In [9]:
load_dotenv("../.env")

True

In [35]:
# === GENERAL CONFIG ===
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_INDEX_NAME = "spa-quz-translation-index"
SRC_LANGUAGE_KEY= "spanish"
TGT_LANGUAGE_KEY = "cuzco quechua"
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"

In [36]:
model = SentenceTransformer(EMBEDDING_MODEL)
pc = Pinecone(api_key=PINECONE_API_KEY)

In [ ]:
'''To create the index in Pinecone, DO NOT RUN IF THE INDEX ALREADY EXISTS. '''

index_name = PINECONE_INDEX_NAME
if index_name not in pc.list_indexes():
    pc.create_index(index_name, dimension=384, metric="cosine", spec=ServerlessSpec(
    cloud="aws",
    region="us-east-1"
  ))

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '6f3408b51a22fa5cdd044d50548c3361', 'Date': 'Thu, 31 Oct 2024 17:20:22 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [38]:
index = pc.Index(PINECONE_INDEX_NAME)
initial_stats = index.describe_index_stats()
print(f"Initial index stats: {initial_stats}")

Initial index stats: {'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 106432}},
 'total_vector_count': 106432}


In [41]:
id = 0
with open('./words.json', 'r') as f:
    words = json.load(f)

    for word in words:
        source_sentence = word[SRC_LANGUAGE_KEY]
        target_sentence = word[TGT_LANGUAGE_KEY]
        source_sentence_embedding = model.encode(source_sentence).tolist()

        print(f"Source sentence: {source_sentence}")
        print(f"Target sentence: {target_sentence}")

        response = index.upsert(vectors=[(str(id), source_sentence_embedding, {"source_language": SRC_LANGUAGE_KEY, 
                                                              "source_sentence": source_sentence, 
                                                              "target_language": TGT_LANGUAGE_KEY, 
                                                              "target_sentence": target_sentence})])
        
        id+=1
        print(f"Upsert response for ID {id}: {response}")

print("Pinecone index created and populated successfully.")

Source sentence: domingo
Target sentence: intichay
Upsert response for ID 1: {'upserted_count': 1}
Source sentence: lunes
Target sentence: killachay
Upsert response for ID 2: {'upserted_count': 1}
Source sentence: martes
Target sentence: antipachay
Upsert response for ID 3: {'upserted_count': 1}
Source sentence: miercoles
Target sentence: qoyllurchay
Upsert response for ID 4: {'upserted_count': 1}
Source sentence: jueves
Target sentence: illapachay
Upsert response for ID 5: {'upserted_count': 1}
Source sentence: viernes
Target sentence: ch'askachay
Upsert response for ID 6: {'upserted_count': 1}
Source sentence: sabado
Target sentence: k'uychichay
Upsert response for ID 7: {'upserted_count': 1}
Source sentence: Enero
Target sentence: Kamay raymi killa
Upsert response for ID 8: {'upserted_count': 1}
Source sentence: Febrero
Target sentence: Hatuy Poqoy killa
Upsert response for ID 9: {'upserted_count': 1}
Source sentence: Marzo
Target sentence: Pawqar waray killa
Upsert response for ID 

In [42]:
stats = index.describe_index_stats()
print(f"Final index stats: {stats}")

Final index stats: {'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 106432}},
 'total_vector_count': 106432}


In [45]:
query_embedding = model.encode("gracias").tolist()
query_response = index.query(
    vector=query_embedding,
    top_k=1,
    include_metadata=True
)
print(f"Query response: {query_response}")

Query response: {'matches': [{'id': '20',
              'metadata': {'source_language': 'spanish',
                           'source_sentence': 'Gracias',
                           'target_language': 'cuzco quechua',
                           'target_sentence': 'Añay'},
              'score': 1.0,
              'values': []}],
 'namespace': '',
 'usage': {'read_units': 6}}
